Copyright (c) Nongnuch Artrith (nartrith@atomistic.net) and Alexander Urban

This Source Code Form is subject to the terms of the Mozilla Public License, v. 2.0. If a copy of the MPL was not distributed with this file, You can obtain one at http://mozilla.org/MPL/2.0/.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the Mozilla Public License, v. 2.0, for more details.

In [ ]:
import functools
import os
import sys

import numpy as np

from scipy.optimize import least_squares

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import howru
from howru.optimize import UOptimizer

In [ ]:
elements = "elements-SCAN+rvv10-energies.csv"
binaries = "binary-oxides-SCAN+rvv10-energies.csv"
ternaries = "ternary-oxides-SCAN+rvv10-energies.csv"
TM_species = ['Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn']

In [ ]:
l2 = 0.01
l2 = 0.00
n_components = 25
n_clusters = 888
U = [0.001, 1.83, 0.93, 2.77, 1.97, 1.83, 2.10, 0.43, 0.001, 0.001]
Jain = None

def objective3(U, opt, l2, R, dE_exp, weight, Jain):
    E_comp = opt.compound_energies(U, Jain=Jain)
    err = (R.dot(E_comp) - dE_exp)*weight
    a = len(weight)/np.sum(weight)
    return np.sqrt(np.mean(err**2))*a + l2*np.linalg.norm(U)

opt = UOptimizer(TM_species, elements, binaries, ternaries, loocv=True)
R, dE_exp, reactions, num_atoms = opt.reaction_matrix()

stdout = sys.stdout
sys.stdout = open("howru.out", "w")

for i in range(len(opt.loocv_comp)):
    comp = opt.loocv_comp[i].composition.formula.replace(" ", "")
    print("\n>>>> LOO-CV Composition: {} <<<<".format(comp))
    opt.reactions = opt.reactions_one_out[i]
    
    # PCA with selected number of PCs
    pca = PCA(n_components=n_components)
    components = pca.fit_transform(R)

    # k-means clustering of the reactions to determine similarities
    kmeans = KMeans(n_clusters=n_clusters).fit(components)
    labels = kmeans.labels_

    # calculate the weight of the reactions based on the frequency of their label
    unique_labels = set(labels)
    counts = {l: labels.tolist().count(l) for l in unique_labels}
    label_weight = [1.0/counts[l] for l in labels]
    
    # optimize U values
    f = functools.partial(objective3, opt=opt, l2=l2, R=R, 
                          dE_exp=dE_exp, weight=label_weight,
                          Jain=Jain)
    results = least_squares(f, U, bounds=(0.0, 5.0), ftol=1.0e-4, xtol=1.0e-3)
  
    # evaluate errors of the left-out reactions
    U_opt = results.x
    opt.print_U_values(U_opt)
    opt.print_Jain()
    opt.reactions = opt.reactions_one_in[i]
    dirname = "loocv-{}".format(comp)
    os.mkdir(dirname)
    opt.eval_energy_errors(
        U_opt, verbose=True,
        fname_binary_oxide_reactions=os.path.join(
            dirname, "oxide-reactions-2.dat"),
        fname_ternary_oxide_reactions=os.path.join(
            dirname, "oxide-reactions-3.dat"),
        fname_binary_o2_reactions=os.path.join(
            dirname, "O2-reactions-2.dat"),
        fname_ternary_o2_reactions=os.path.join(
            dirname, "O2-reactions-3.dat"),
        fname_binary_oxide_formations=os.path.join(
            dirname, "formation-energies-2.dat"),
        fname_ternary_oxide_formations=os.path.join(
            dirname, "formation-energies-3.dat"))
    
    opt.reactions = opt.all_reactions
    
sys.stdout = stdout